In [1]:
#coding=utf-8
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

def load_data():
    df = pd.read_csv("Data/train.csv.gz")
    return df

def eval_matric(y_true, y_prob):
    print("true p ratio", sum(y_true)/ len(y_true))
    y_pred = [1 if i> 0.5 else 0 for i in y_prob]
    print("recall ratio", sum(y_pred)/float(sum(y_true)))

    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
        gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    print("gini:", gini)
    return gini

In [2]:
from keras import optimizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input,Dense, concatenate,Dropout,BatchNormalization,Activation,Flatten,Add, Conv2D, Dot, dot
from keras.layers import RepeatVector, merge, Subtract, Lambda, Multiply, Embedding, Concatenate, Reshape, DepthwiseConv2D
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.engine.topology import Layer
from sklearn.metrics import roc_auc_score, recall_score
    
def CIN(x, conv_layer):
    cat_output_expand = Lambda(lambda x: K.expand_dims(x, axis = 2))(x)

    # shape: -1, 1, feature_size, dim
    x_0 = Lambda(lambda x: K.permute_dimensions(x, (0,3,2,1)))(cat_output_expand)
    x_next = cat_output_expand

    cin_output = []
    for layer in conv_layer:
        x = Lambda(lambda x: K.permute_dimensions(x, (0,3,1,2)))(x_next)
        z_0 = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=(2,3)))([x_0, x])
        z_1 = Lambda(lambda x: K.permute_dimensions(x, (0,2,3,1)))(z_0)

        x_next_list = []
        pooling_output_list = []
        for index in range(layer):
            #depth_multiplier=3
            output = DepthwiseConv2D((int(z_1.shape[1]), int(z_1.shape[2])))(z_1)
            #output = Conv2D(3, (int(z_1.shape[1]), int(z_1.shape[2])) )(z_1)
            output = Lambda(lambda x: K.squeeze(x, 2))(output)
            pooling_output = Lambda(lambda x: K.sum(output, axis = 2))(output)
            pooling_output_list.append(pooling_output)
            x_next_list.append(output)
        x_next = Concatenate(axis = 1)(x_next_list)
        x_next = Lambda(lambda x: K.expand_dims(x, axis = 2))(x_next)

        x_pooling = Concatenate(axis = 1)(pooling_output_list)
        cin_output.append(x_pooling)

    cin_output =  Concatenate(axis = 1)(cin_output)
    cin_output = Dense(1, activation='linear')(cin_output)
    return cin_output
    
class XDeepFM():
    def __init__(self, sparse_features, dense_features, sparse_label_dict, hidden_layer, cov_layer, embed_dim):
        self.sparse_features = sparse_features
        self.dense_features = dense_features
        self.sparse_label_dict = sparse_label_dict
        self.hidden_layer = hidden_layer
        self.embed_dim = embed_dim
        self.sparse_label_dict = sparse_label_dict
        self.con_layer = cov_layer
        
    def fit(self, train, y_train, n_epoch=100, n_batch = 1000):
        cat_input = []
        cat_output = []
        for col in self.sparse_features:
            input = Input(shape= (1,))
            cat_input.append(input)
            emb = Embedding(self.sparse_label_dict[col], self.embed_dim, input_length =1 ,trainable = True)(input)
            cat_output.append(emb)
         
        cat_output = Concatenate(axis=1)(cat_output)
        
        #first_order = Added_Weights(use_bias = True)(cat_output)
        first_order = Flatten()(cat_output)
        first_order = Dense(1, activation='linear')(first_order)
        
        # 需要使用lambda 层封装Backend 的函数操作
        #first_order = Lambda(lambda x: K.sum(x, axis =1, keepdims=True))(first_order)
        cin_output = CIN(cat_output, self.con_layer)
        '''
        cat_output_expand = Lambda(lambda x: K.expand_dims(x, axis = 2))(cat_output)

        # shape: -1, 1, feature_size, dim
        x_0 = Lambda(lambda x: K.permute_dimensions(x, (0,3,2,1)))(cat_output_expand)
        x_next = cat_output_expand
    
        cin_output = []
        for layer in self.con_layer:
            x = Lambda(lambda x: K.permute_dimensions(x, (0,3,1,2)))(x_next)
            z_0 = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=(2,3)))([x_0, x])
            z_1 = Lambda(lambda x: K.permute_dimensions(x, (0,2,3,1)))(z_0)
            
            x_next_list = []
            pooling_output_list = []
            for index in range(layer):
                output = Conv2D(3, (int(z_1.shape[1]), int(z_1.shape[2])) )(z_1)
                output = Lambda(lambda x: K.squeeze(x, 2))(output)
                pooling_output = Lambda(lambda x: K.sum(output, axis = 2))(output)
                pooling_output_list.append(pooling_output)
                x_next_list.append(output)
            x_next = Concatenate(axis = 1)(x_next_list)
            x_next = Lambda(lambda x: K.expand_dims(x, axis = 2))(x_next)
            
            x_pooling = Concatenate(axis = 1)(pooling_output_list)
            cin_output.append(x_pooling)
        
        cin_output =  Concatenate(axis = 1)(cin_output)
        cin_output = Dense(1, activation='linear')(cin_output)
        '''
        
        dense_input = Input(shape = (len(self.dense_features), ))
        
        dnn_input = Concatenate(axis=1)([Flatten()(cat_output), dense_input])
        #dnn_input = dense_input 
        dnn_output = dnn_input 
        for layer in self.hidden_layer:
            dnn_output  = BatchNormalization()(dnn_output)
            dnn_output  = Dense(layer, activation='relu')(dnn_output)
            dnn_output  = Dropout(0.2)(dnn_output)
        dnn_output = Dense(1, activation='linear')(dnn_output)
        
        #output  = Concatenate(axis=1)([first_order, second_order, dnn_output])
        output  = Add()([first_order, cin_output, dnn_output])
        output = Dense(1, activation='sigmoid')(output)
        model = Model(inputs = cat_input + [dense_input], outputs=output)
        print("---starting the training---")
        model.compile(
            optimizer="adam",
            loss='binary_crossentropy',
            metrics=["accuracy"]
        )
        print(model.summary())
        model.fit([train[f] for f in self.sparse_features] + [train[self.dense_features]], y_train, epochs = n_epoch, batch_size= n_batch)
        self.model = model
        
    def predict(self, test):
        y_pred = self.model.predict([test[f] for f in self.sparse_features] +  [test[self.dense_features]])
        return y_pred
    
    def evaluate(self, test, y_test, metrics):
        #loss, accuracy = self.model.evaluate([test[f] for f in self.sparse_features] +  [test[self.dense_features]], y_test)
        #print('\n', 'test accuracy:', accuracy)
        y_pred = self.predict(test)
        return metrics(y_test, y_pred)

Using TensorFlow backend.


In [3]:
print("---loading and preprocessing the data---")
data = load_data()
data = data.set_index("id")
target = data['target']
data.drop(['target'], axis=1, inplace=True)
#data = preprocess(data)

---loading and preprocessing the data---


In [4]:
from util import *

data, sparse_features, dense_features = recognize_feature(data)
print(len(dense_features),len(sparse_features))

sparse_label_dict = dict()
for f in sparse_features:
    sparse_label_dict[f] = data[f].max()

print(data.shape)
#data = one_hot_for_sparse(data, sparse_features)
#data = scalar_for_dense(data, dense_features)
print(data.shape)

sparse : unique sum  376
27 30
(595212, 57)
(595212, 57)


In [5]:
print(sparse_label_dict)

{'ps_ind_01': 7, 'ps_ind_02_cat': 4, 'ps_ind_03': 11, 'ps_ind_04_cat': 2, 'ps_ind_05_cat': 7, 'ps_ind_14': 4, 'ps_ind_15': 13, 'ps_car_01_cat': 12, 'ps_car_02_cat': 2, 'ps_car_03_cat': 2, 'ps_car_04_cat': 9, 'ps_car_05_cat': 2, 'ps_car_06_cat': 17, 'ps_car_07_cat': 2, 'ps_car_08_cat': 1, 'ps_car_09_cat': 5, 'ps_car_10_cat': 2, 'ps_car_11_cat': 103, 'ps_car_11': 4, 'ps_calc_04': 5, 'ps_calc_05': 6, 'ps_calc_06': 10, 'ps_calc_07': 9, 'ps_calc_08': 10, 'ps_calc_09': 7, 'ps_calc_10': 25, 'ps_calc_11': 19, 'ps_calc_12': 10, 'ps_calc_13': 13, 'ps_calc_14': 23}


In [6]:
train, test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

In [7]:
train = train.reset_index().drop("id", axis =1)
test = test.reset_index().drop("id", axis =1)
y_train = y_train.reset_index().drop("id", axis =1)
y_test = y_test.reset_index().drop("id", axis =1)

%%time
model = XDeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , [25, 20, 20], 3)
model.fit(train, y_train, 100, 1000)
model.evaluate(test, y_test, roc_auc_score)

%%time
from Keras.XDeepFM import XDeepFM
model = XDeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , [25, 20, 20], 3)
model.fit(train, y_train, 50, 1000)
model.evaluate(test, y_test, roc_auc_score)

%%time
from Tensorflow.TFModel import *
model =TFModel(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , 3)
model.fit(train, y_train, 100, 1000)
model.evaluate(test, y_test, roc_auc_score)

%%time
from Keras.DeepFM import *
model = DeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , 3)
model.fit(train, y_train, 100, 1000)
model.evaluate(test, y_test, roc_auc_score)

In [10]:
%%time
from Tensorflow.DeepFM import DeepFM
model = DeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , 3)
model.fit(train, y_train, 10, 1000)
model.evaluate(test, y_test, roc_auc_score)

epoch 0:
the times of training is 0, and the loss is 0.693991
the times of training is 100, and the loss is 0.19673306


KeyboardInterrupt: 